<a href="https://colab.research.google.com/github/akshithamaddi/akshithamaddi_INFO5731/blob/master/Bertfortweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#topic Modeling
import pandas as pd
topicm = pd.read_csv("2020-3-new.csv",encoding="ISO-8859-1")
header = ["clean_tweet_text"]
topicm.to_csv('output.csv', columns = header)

In [ ]:
pip install pyLDAvis==2.1.2

     |████████████████████████████████| 1.6MB 16.5MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=d69e7b705999bf2a2d84c7e8502dda02bf850360da2f90f21191dd05cecd0493
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
pip install "gensim==3.8.1"

     |████████████████████████████████| 24.2MB 44.2MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
import nltk;
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
data = topicm.clean_tweet_text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

['sources telling early friday USER_MENTION ask employees lie home chinavirus chinesevirus indiafightscoronavirus']


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['sources', 'telling', 'early', 'friday', 'user_mention', 'ask', 'employees', 'lie', 'home', 'chinavirus', 'chinesevirus']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['sources', 'telling', 'early', 'friday', 'user_mention', 'ask', 'employees', 'lie', 'home', 'chinavirus', 'chinesevirus']


In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['source', 'tell', 'early', 'ask', 'employee', 'lie', 'home', 'chinavirus']]


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ask', 1),
  ('chinavirus', 1),
  ('early', 1),
  ('employee', 1),
  ('home', 1),
  ('lie', 1),
  ('source', 1),
  ('tell', 1)]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=37, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
coherence = []
perplexity = []
for k in range(5,75):
  print('Round: '+str(k))
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           chunksize=10000,
                                           passes=10,
                                           iterations=200,
                                          eval_every = None) 
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')                                              
  coherence.append([k,coherence_model_lda.get_coherence()])
  doc_lda = lda_model[corpus]
  perplexity.append(lda_model.log_perplexity(corpus))

In [ ]:
print(coherence)
print(perplexity)

[[5, 0.34987340537719935], [6, 0.3140970390893331], [7, 0.30507482771884853], [8, 0.3335139438150748], [9, 0.34754868298642067], [10, 0.37886358959704936], [11, 0.34323949101170054], [12, 0.3586346535462459], [13, 0.3624084517384147], [14, 0.3563272054975001], [15, 0.3664047568169619], [16, 0.34771817288818574], [17, 0.3653916947241706], [18, 0.33826525306672445], [19, 0.37896852942726433], [20, 0.3829741843081077], [21, 0.38978701689730016], [22, 0.36480234767479036], [23, 0.3772903441634384], [24, 0.3764081043428564], [25, 0.38988774409395477], [26, 0.39002701290529623], [27, 0.4000362447810887], [28, 0.3899791856721394], [29, 0.38197406770859443], [30, 0.38473864482978926], [31, 0.4023445432567965], [32, 0.4051956589600385], [33, 0.41796591751033346], [34, 0.41555616084557867], [35, 0.41512456899761196], [36, 0.41929056175156576], [37, 0.4038544424946787], [38, 0.4327979527706474], [39, 0.44497599016415573], [40, 0.43754315445588643], [41, 0.42691854513963845], [42, 0.44085876813972

In [ ]:
column_names = ["num topics","Coherence"]
df = pd.DataFrame(coherence, columns=column_names)
df['perplexity'] = perplexity
df

,num topics,Coherence,perplexity
0,5,0.349873,-6.438674
1,6,0.314097,-6.451023
2,7,0.305075,-6.391372
3,8,0.333514,-6.481422
4,9,0.347549,-6.522799
...,...,...,...
67,72,0.512050,-16.892637
68,73,0.495329,-17.023939
69,74,0.512100,-17.301782
70,75,0.484277,-17.314453


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df['num topics'], y=df['Coherence'], name="coherence"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['num topics'], y=df['perplexity'], name="perplexity"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Coherence and preplexity"
)

# Set x-axis title
fig.update_xaxes(title_text="Num topics")

# Set y-axes titles
fig.update_yaxes(title_text="Coherence", secondary_y=False)
fig.update_yaxes(title_text="Perplexity", secondary_y=True)

fig.show()

In [ ]:
import pandas as pd
df = pd.read_csv("2020-3-new.csv",encoding="ISO-8859-1")
header = ["clean_tweet_text"]
df.to_csv('output.csv', columns = header)

In [9]:
pip install bertopic

     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 81kB 4.1MB/s 
     |████████████████████████████████| 6.4MB 6.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2MB 37.1MB/s 
     |████████████████████████████████| 2.2MB 43.9MB/s 
     |████████████████████████████████| 1.2MB 46.9MB/s 
     |████████████████████████████████| 870kB 32.7MB/s 
     |████████████████████████████████| 3.3MB 44.8MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311690 sha256=c55ae25432eadf500fd8fd2dd16a205485c7963a358b28a0517d3dba2c28e426
  Stored in directory: /root/.cache/pip/wheels/42/63/fb/314ad6c3b270887a3ecb588b8e5aac50b0fad38ff89bb6dff2
Successfully built hdbscan
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=a8aed46c8716a9148cea40af0b8e4b4288498

In [10]:
pip install bertopic[visualization]

     |████████████████████████████████| 13.2MB 328kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [11]:
pip install bertopic[flair]

     |████████████████████████████████| 450kB 4.3MB/s 
     |████████████████████████████████| 798kB 8.2MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 983kB 47.7MB/s 
  Created wheel for segtok: filename=segtok-1.5.10-cp37-none-any.whl size=25019 sha256=aa10cd45a98b965bc732de475f48a0c005b429edefddeebc6455698e23b6ad45
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
  Created wheel for sqlitedict: filename=sqlitedict-1.7.0-cp37-none-any.whl size=14376 sha256=73d0a15be789984d5f10f7877addbb5136f7e3f23bedfc197252a9eb22780710
  Stored in directory: /root/.cache/pip/wheels/cf/c6/4f/2c64a43f041415eb8b8740bd80e15e92f0d46c5e464d8e4b9b
  Created wheel for mpld3: filename=mpld3-0.3-cp37-none-any.whl size=116679 sha256=4e42b4f5cb3e21cd8518e9798d5653747cfe93988c0753bdf4dedaf1421213c6
  Stored in directory: /root/.cache/pip/wheels/c0/4

In [ ]:
from bertopic import BERTopic

df1= pd.read_csv("output.csv")
docs = df1['clean_tweet_text']
topic_model = BERTopic()
topics, probabilities = topic_model.fit_transform(docs)

In [ ]:
new_topics, new_probs = topic_model.reduce_topics(docs, topics, probabilities, nr_topics=38)

In [ ]:
topic_model.get_topics()

{-1: [('covid19', 0.016650093473956604),
  ('coronavirus', 0.01373414803035172),
  ('wuhanvirus', 0.012767122327502952),
  ('coronavirusoutbreak', 0.012047448566644353),
  ('covid2019', 0.011836409167569704),
  ('quarantinelife', 0.00868337406473148),
  ('corona', 0.008490167208637045),
  ('chinaliedpeopledied', 0.006648136626478216),
  ('wuhancoronavirus', 0.006457812810471358),
  ('socialdistancing', 0.006236408033187403)],
 0: [('afraid', 0.12193361694433838),
  ('journalists', 0.12113263521124543),
  ('propaganda', 0.10402103355841139),
  ('muttering', 0.0004843938430637791),
  ('complymost', 0.0004843938430637791),
  ('backstab', 0.0004843938430637791),
  ('india', 0.00047731172753200335),
  ('trap', 0.0003944311301434361),
  ('consumer', 0.00039146668713258204),
  ('market', 0.00025669561485574144)],
 18: [('karimnagar', 0.19627739793318477),
  ('ranch', 0.19608853216765898),
  ('erode', 0.19583775398992392),
  ('smoking', 0.19565044634240006),
  ('pain', 0.18865434603393788),
  

In [ ]:
topic_model.visualize_topics()